# Understanding IFC Files: Building Code Compliance Exercises

## What is IFC?

**IFC (Industry Foundation Classes)** is a standard data format for building information. It stores geometric, spatial, and semantic information about buildings in a structured way. Think of it as a database format for architectural and engineering data.

Industries use IFC to:
- Share building models across different software
- Extract specific information (spaces, materials, systems)
- Validate designs against building codes
- Run simulations and analysis

## Resources

Before starting, explore these resources to understand IFC better:

- **IFC Knowledge Base**: https://notebooklm.google.com/notebook/0925c2a1-519b-40a8-aca4-1e832d219f3c
- **BuildingSmart (IFC Standard)**: https://www.buildingsmart.org/standards/bsi-standards/industry-foundation-classes/
- **IfcOpenShell Python Docs**: https://docs.ifcopenshell.org/ifcopenshell-python.html
- **Catalan Building Code Reference**: https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e (for exercises 1 & bonus)

This notebook uses **IfcOpenShell**, a Python library for reading and writing IFC files.

## Setup: Load and Explore IFC Files

First, let's install IfcOpenShell and load the duplex apartment model.

In [5]:
# Install IfcOpenShell (run this once)
import subprocess
import sys

try:
    import ifcopenshell
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ifcopenshell", "-q"])
    import ifcopenshell

# Import other useful libraries
import json
from pathlib import Path
from collections import defaultdict

# Load the IFC file
ifc_file_path = Path("assets/duplex.ifc")
ifc = ifcopenshell.open(ifc_file_path)

print(f"✓ IFC file loaded: {ifc_file_path}")
print(f"✓ IFC Schema: {ifc.schema}")

✓ IFC file loaded: assets\duplex.ifc
✓ IFC Schema: IFC2X3


### What's Inside the File?

Let's explore the structure of the IFC model:

In [6]:
# Get all unique entity types in the model
all_entities = list(ifc)
entity_types = defaultdict(int)
for entity in all_entities:
    entity_types[entity.is_a()] += 1

# Show entity counts
print("Entity types in this model:")
print("-" * 40)
for entity_type in sorted(entity_types.keys()):
    count = entity_types[entity_type]
    print(f"  {entity_type}: {count}")

print(f"\nTotal entities: {len(all_entities)}")

# Find basic info
building = ifc.by_type("IfcBuilding")[0] if ifc.by_type("IfcBuilding") else None
if building:
    print(f"\nBuilding name: {building.Name}")
    print(f"Building description: {building.Description}")

Entity types in this model:
----------------------------------------
  IfcApplication: 1
  IfcArbitraryClosedProfileDef: 102
  IfcArbitraryOpenProfileDef: 184
  IfcArbitraryProfileDefWithVoids: 18
  IfcAxis2Placement2D: 397
  IfcAxis2Placement3D: 1279
  IfcBeam: 8
  IfcBooleanClippingResult: 7
  IfcBuilding: 1
  IfcBuildingStorey: 4
  IfcCartesianPoint: 8520
  IfcCartesianTransformationOperator3D: 167
  IfcCircle: 96
  IfcCircleProfileDef: 8
  IfcColourRgb: 43
  IfcCompositeCurve: 44
  IfcCompositeCurveSegment: 322
  IfcConnectedFaceSet: 235
  IfcConnectionSurfaceGeometry: 265
  IfcConversionBasedUnit: 1
  IfcCovering: 13
  IfcCurveBoundedPlane: 81
  IfcCurveStyle: 31
  IfcCurveStyleFont: 19
  IfcCurveStyleFontPattern: 57
  IfcDimensionalExponents: 1
  IfcDirection: 134
  IfcDoor: 14
  IfcDoorLiningProperties: 6
  IfcDoorStyle: 6
  IfcDraughtingPreDefinedCurveFont: 1
  IfcElementQuantity: 21
  IfcExtrudedAreaSolid: 421
  IfcFace: 4486
  IfcFaceBasedSurfaceModel: 40
  IfcFaceBound: 60
 

## Example: Extract and Display All IFC Spaces

An **IfcSpace** represents a room or area in the building. Each space has properties like name, area, and volume. Here's how to extract them:

In [7]:
# Get all spaces
spaces = ifc.by_type("IfcSpace")

print(f"Found {len(spaces)} spaces in the building\n")
print("=" * 60)

for i, space in enumerate(spaces, 1):
    # Extract properties
    name = space.Name if space.Name else "Unnamed"
    
    # Try to get area and volume
    area = None
    volume = None
    
    if hasattr(space, 'Quantities') and space.Quantities:
        for quantity in space.Quantities.Quantities:
            if hasattr(quantity, 'Name'):
                if quantity.Name == "NetFloorArea" and hasattr(quantity, 'AreaValue'):
                    area = quantity.AreaValue
                elif quantity.Name == "GrossVolume" and hasattr(quantity, 'VolumeValue'):
                    volume = quantity.VolumeValue
    
    # Format output
    print(f"{i}. {name}")
    if area:
        print(f"   Area: {area:.2f} m²")
    if volume:
        print(f"   Volume: {volume:.2f} m³")
    print()

print("=" * 60)

Found 21 spaces in the building

1. A101

2. A201

3. B104

4. B101

5. B201

6. A205

7. B205

8. A105

9. B105

10. R301

11. B102

12. A102

13. A103

14. B103

15. B204

16. A204

17. A104

18. B203

19. A202

20. B202

21. A203



## Exercise 1: Building Code Compliance Checker (Catalonia)

**Objective:** Write a function that validates spaces against Catalonia building code requirements.

**Reference:** [Catalan Building Code](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Key Requirements to Check

Based on the Catalan building code, residential spaces should meet these minimum standards:

| Space Type | Min Height | Min Area | Purpose |
|---|---|---|---|
| Living Room | 2.6 m | 16 m² | Main living area |
| Bedroom | 2.6 m | 9 m² | Single occupancy |
| Kitchen | 2.6 m | 8 m² | Cooking/dining |
| Bathroom | 2.3 m | 4 m² | Hygiene |
| Corridor | 2.3 m | 1.5 m² | Circulation |

### Your Task

Write a function `check_space_compliance(spaces)` that:

1. **Identifies** each space type (you can infer from the name or classify them)
2. **Extracts** the height and area properties from each space
3. **Compares** against the requirements above
4. **Reports** which spaces pass/fail and why
5. **Returns** a summary with compliance status

**Hints:**
- Height can be extracted from the Z-coordinate range of the space geometry
- Area is usually stored in space.Quantities as "NetFloorArea"
- Space names often indicate their type (e.g., "Master Bedroom", "Kitchen")

### Starter Code

In [ ]:
# Exercise 1: Implement your solution here
# Copy the starter code below and complete the function

def check_space_compliance(spaces):
    """
    Check each space for Catalan building code compliance.
    
    Args:
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Compliance report
    """
    
    requirements = {
        "Living Room": {"min_height": 2.6, "min_area": 16},
        "Bedroom": {"min_height": 2.6, "min_area": 9},
        "Kitchen": {"min_height": 2.6, "min_area": 8},
        "Bathroom": {"min_height": 2.3, "min_area": 4},
        "Corridor": {"min_height": 2.3, "min_area": 1.5},
    }
    
    report = {
        "passed": [],
        "failed": [],
        "warnings": []
    }
    
    def extract_area(space):
        """Extract floor area from space quantities"""
        if hasattr(space, 'Quantities') and space.Quantities:
            for quantity in space.Quantities.Quantities:
                if hasattr(quantity, 'Name') and quantity.Name == "NetFloorArea":
                    if hasattr(quantity, 'AreaValue'):
                        return quantity.AreaValue
        return None

    def extract_height(space):
        """Extract height from space geometry (Z-coordinate range)"""
        if hasattr(space, 'Representation') and space.Representation:
            for rep in space.Representation.Representations:
                if hasattr(rep, 'Items') and rep.Items:
                    z_coords = []
                    for item in rep.Items:
                        if hasattr(item, 'Points'):  # PolyLoop or similar
                            for point in item.Points:
                                if hasattr(point, 'Coordinates') and len(point.Coordinates) >= 3:
                                    z_coords.append(point.Coordinates[2])
                    if z_coords:
                        return max(z_coords) - min(z_coords)
        return None

    def classify_space_type(space_name, area):
        """Classify space type based on name and area"""
        name_lower = space_name.lower()

        # Name-based classification
        if "bed" in name_lower:
            return "Bedroom"
        elif "bath" in name_lower or "wc" in name_lower:
            return "Bathroom"
        elif "kit" in name_lower:
            return "Kitchen"
        elif "corr" in name_lower or "hall" in name_lower:
            return "Corridor"

        # Area-based classification (fallback)
        if area:
            if area < 5:
                return "Bathroom"
            elif area < 8:
                return "Bathroom"
            elif area < 12:
                return "Kitchen"
            elif area < 18:
                return "Bedroom"
            else:
                return "Living Room"

        return "Living Room"  # Default

    # Process each space
    for space in spaces:
        space_name = space.Name if space.Name else "Unnamed"
        area = extract_area(space)
        height = extract_height(space)

        # Identify space type
        space_type = classify_space_type(space_name, area)

        # Get requirements for this type
        if space_type not in requirements:
            report["warnings"].append({
                "name": space_name,
                "type": space_type,
                "reason": "Unknown space type"
            })
            continue

        req = requirements[space_type]

        # Check compliance
        passed = True
        failures = []

        if area is None:
            report["warnings"].append({
                "name": space_name,
                "type": space_type,
                "reason": "Could not extract area"
            })
            passed = False
        elif area < req["min_area"]:
            passed = False
            failures.append(f"Area {area:.2f}m² < {req['min_area']}m² (required)")

        if height is None:
            report["warnings"].append({
                "name": space_name,
                "type": space_type,
                "reason": "Could not extract height"
            })
            passed = False
        elif height < req["min_height"]:
            passed = False
            failures.append(f"Height {height:.2f}m < {req['min_height']}m (required)")

        # Add to appropriate list
        entry = {
            "name": space_name,
            "type": space_type,
            "area": area,
            "height": height,
        }

        if passed and area is not None and height is not None:
            report["passed"].append(entry)
        elif area is not None and height is not None:
            entry["failures"] = failures
            report["failed"].append(entry)

    return report


# Test your function (uncomment when ready)
result = check_space_compliance(spaces)
print(f"Passed: {len(result['passed'])} spaces")
print(f"Failed: {len(result['failed'])} spaces")

In [ ]:
# Display detailed compliance report
result = check_space_compliance(spaces)

print("=" * 70)
print("COMPLIANCE REPORT - CATALAN BUILDING CODE")
print("=" * 70)

# Summary
print(f"\nSUMMARY:")
print(f"  Passed: {len(result['passed'])} spaces")
print(f"  Failed: {len(result['failed'])} spaces")
print(f"  Warnings: {len(result['warnings'])} spaces")

# Detailed passed spaces
if result['passed']:
    print(f"\n{'PASSED SPACES:':-^70}")
    for space in result['passed']:
        print(f"\n  {space['name']} ({space['type']})")
        print(f"    Area: {space['area']:.2f} m² | Height: {space['height']:.2f} m")

# Detailed failed spaces
if result['failed']:
    print(f"\n{'FAILED SPACES (Non-Compliant):':-^70}")
    for space in result['failed']:
        print(f"\n  {space['name']} ({space['type']})")
        print(f"    Area: {space['area']:.2f} m² | Height: {space['height']:.2f} m")
        for failure in space.get('failures', []):
            print(f"    ✗ {failure}")

# Warnings
if result['warnings']:
    print(f"\n{'WARNINGS (Incomplete Data):':-^70}")
    for warning in result['warnings']:
        print(f"\n  {warning['name']} ({warning['type']})")
        print(f"    ⚠ {warning['reason']}")

print("\n" + "=" * 70)

## Exercise 2: Window Detection and Compliance Verification

**Objective:** Find all windows in the model and verify they meet natural light and ventilation requirements.

**Reference:** [Catalan Building Code](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Key Requirements

Residential spaces must have:
- **Minimum window area** = 1/8 of floor area (12.5%)
- **Minimum window dimensions** = 60cm width, 100cm height (for single opening)
- Living areas should have direct natural light

### Your Task

Write a function `analyze_window_compliance(ifc_model, spaces)` that:

1. **Finds all IfcWindow** entities in the model
2. **Links windows to spaces** (which room is each window in?)
3. **Extracts properties**: dimensions, area, orientation
4. **Calculates window-to-floor ratio** for each space
5. **Reports compliance** for each space with windows

**Hints:**
- Windows are `IfcWindow` entities
- To link windows to spaces, check spatial containment relationships
- Window area can be calculated from the frame/pane dimensions
- You may need to examine the geometric representation

### Starter Code

In [ ]:
# Exercise 2: Implement your solution here

def analyze_window_compliance(ifc_model, spaces):
    """
    Analyze windows in each space and check compliance.
    
    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Report with window analysis and compliance status
    """
    
    windows = ifc_model.by_type("IfcWindow")
    
    report = {
        "total_windows": len(windows),
        "windows_by_space": {},
        "compliance_status": {}
    }
    
    print(f"Found {len(windows)} windows in the model")
    
    # TODO: Extract window properties (name, dimensions)
    # TODO: Group windows by the space they're in
    # TODO: Calculate total window area per space
    # TODO: Calculate window-to-floor ratio
    # TODO: Check if the requirement is met
    
    return report


# Test your implementation (uncomment when ready)
analysis = analyze_window_compliance(ifc, spaces)

Found 24 windows in the model


## Bonus Exercise: Fire Safety Route Analysis

**Objective:** Find the longest evacuation route within the apartment and verify it meets fire safety requirements.

**Difficulty:** Advanced

**Reference:** [Catalan Building Code - Fire Safety Section](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Fire Safety Requirements

According to the Catalan building code:
- **Maximum travel distance** to exit: ≤ 25-30 m (depending on building type)
- **Minimum corridor width**: 1.2 m
- **Minimum door width**: 0.8 m (for exits)
- **No dead-end corridors** longer than 10 m

### Your Task

Write a function `analyze_evacuation_routes(ifc_model, spaces)` that:

1. **Builds a spatial graph** of the apartment (rooms as nodes, doors/openings as connections)
2. **Calculates distances** between spaces (using area/perimeter as proxy)
3. **Finds the longest route** from any point to the nearest exit
4. **Validates** the route against fire safety requirements
5. **Identifies bottlenecks** (narrow corridors, small doors)

**Hints:**
- Think of spaces as nodes and doors as edges in a graph
- Use BFS/DFS to find longest paths
- Door dimensions can indicate width constraints
- Consider calculating distances based on space geometry
- This is a simplified model - real analysis would use detailed geometry

### Starter Code

```python
def analyze_evacuation_routes(ifc_model, spaces):
    """
    Analyze evacuation routes and fire safety compliance.
    
    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Fire safety analysis and recommendations
    """
    
    # Get all doors (potential connections between spaces)
    doors = ifc_model.by_type("IfcDoor")
    
    analysis = {
        "total_spaces": len(spaces),
        "total_doors": len(doors),
        "longest_route": None,
        "longest_distance": 0,
        "safety_issues": [],
        "compliant": False
    }
    
    print(f"Analyzing {len(spaces)} spaces with {len(doors)} doors")
    
    # TODO: Build spatial connectivity graph
    # TODO: Find longest evacuation path
    # TODO: Check corridor widths and door dimensions
    # TODO: Validate against requirements
    # TODO: Report issues and recommendations
    
    return analysis


# Run the analysis (uncomment when ready)
# fire_analysis = analyze_evacuation_routes(ifc, spaces)
```

In [ ]:
# Bonus Exercise: Implement your solution here

from collections import deque, defaultdict
import math

def analyze_evacuation_routes(ifc_model, spaces):
    """
    Analyze evacuation routes and fire safety compliance.
    
    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Fire safety analysis and recommendations
    """
    
    # Get all doors (potential connections between spaces)
    doors = ifc_model.by_type("IfcDoor")
    
    analysis = {
        "total_spaces": len(spaces),
        "total_doors": len(doors),
        "longest_route": None,
        "longest_distance": 0,
        "safety_issues": [],
        "compliant": False
    }
    
    print(f"Analyzing {len(spaces)} spaces with {len(doors)} doors")
    
    # TODO: Build a graph where:
    #   - Each space is a node
    #   - Each door connecting spaces is an edge
    
    # TODO: Calculate distance/cost for each connection
    
    # TODO: Use BFS or Dijkstra to find longest path from any space to nearest exit
    
    # TODO: Check if longest_distance <= 25m (requirement)
    
    # TODO: Extract and check door dimensions
    
    # TODO: Report safety issues and recommendations
    
    return analysis


# Test your solution (uncomment when ready)
# fire_analysis = analyze_evacuation_routes(ifc, spaces)
# if fire_analysis['compliant']:
#     print("✓ Evacuation routes are compliant")
# else:
#     print("✗ Fire safety issues found:")
#     for issue in fire_analysis['safety_issues']:
#         print(f"  - {issue}")

## Useful IFC Concepts

### Common Entity Types

- **IfcSpace**: A room, area, or zone in the building
- **IfcWindow**: Windows for light and ventilation  
- **IfcDoor**: Doors, openings, access points
- **IfcWall**: Boundary elements
- **IfcBuildingElement**: General building components
- **IfcElement**: Physical building pieces with properties

### Accessing Properties

```python
# Get all entities of a type
elements = ifc.by_type("IfcWindow")

# Access properties
entity.Name                    # String name
entity.Description             # Text description
entity.ObjectPlacement        # Location/coordinates
entity.Representation         # Geometry data
entity.QuantityInSpace         # Calculate-derived quantities

# Common relationships
entity.HasProperties           # Get properties
entity.BoundedBy               # Spatial boundaries
entity.HostedBy                # Connection relationships
```

### Helpful Methods

```python
# Query by GUID
entity = ifc.by_id(guid)

# Filter by property value
elements = ifc.by_attribute("Name", "Kitchen")

# Get all instances of a type
spaces = ifc.by_type("IfcSpace")

# Check entity type
if space.is_a("IfcSpace"):
    print("This is a space")
```

## Resources for Help

- **IFC Knowledge Base**: https://notebooklm.google.com/notebook/0925c2a1-519b-40a8-aca4-1e832d219f3c
- **IfcOpenShell Documentation**: https://docs.ifcopenshell.org/ifcopenshell-python.html
- **BuildingSmart Standards**: https://www.buildingsmart.org/
- **Catalan Building Code**: https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e

## Next Steps

After completing these exercises, you'll have learned:
- ✓ How to load and explore IFC files
- ✓ How to extract spatial and building data
- ✓ How to validate designs against building codes
- ✓ How to work with doors, windows, and routes

These skills apply to real-world AEC (Architecture, Engineering, Construction) workflows.